In [21]:
import numpy as np
import numpy.linalg as LA
from scipy import integrate
import inspect

## Метод колокації для інтегрального рівняння Фредгольма другого роду

Маємо інтегральне рівняння вигляду

$$
\varphi(x)-\lambda \int_{a}^{b} \varphi(y) K(x, y) d y = f(x), \quad x \in[a, b]
$$

Наближений розв'язок будемо шукати у вигляді

$$
\tilde{\varphi}(x)=\sum_{j=1}^{n} c_{j} \gamma_{j}(x)
$$

де $c_j$ - невідомі константи, які потрібно знайти, а $\gamma_j$ - задані лінійнонезалежні функції (в нашому випадку кусково аналітичні), $j = 1, ..., n$.

Тоді отримаємо такий вираз

$$
\sum_{j=1}^{n} c_{j}\left[\gamma_{j}(x)-\lambda \int_{a}^{b} K(x, y) \gamma_{j}(y) d y\right]=f(x)
$$

Розглянувши його на множині точок $x_i$, таких, що
$$
a \leqslant x_{1}<x_{2}<\ldots<x_{m} \leqslant b
$$

отримаємо СЛАР відносно невідомих $с_j$, $j = 1, ..., n$. 

In [4]:
a, b = -1, 1
K = lambda x, y: x*y + x**2
f = lambda x: 1
lambd = 1
h = 0.1

In [5]:
xx = np.arange(a, b + 0.000001, h).round(3).tolist()
n = len(xx)
A = np.zeros((n,n)).tolist()
xx

[-1.0,
 -0.9,
 -0.8,
 -0.7,
 -0.6,
 -0.5,
 -0.4,
 -0.3,
 -0.2,
 -0.1,
 -0.0,
 0.1,
 0.2,
 0.3,
 0.4,
 0.5,
 0.6,
 0.7,
 0.8,
 0.9,
 1.0]

In [6]:
def basicFunction(j, x, xx=xx, h=h):
  '''j=0...n-1, базис на [xx[j], xx[j+1]]'''
  n = len(xx)-1
  #P0 = 1
  
  #P1 = lambda y: (y - xx[j])/h

  #print(x)
  
  if j >=1 and xx[j-1] <= x <= xx[j]:
    return (x - xx[j-1])/h
  elif j <= n-1 and xx[j] <= x <= xx[j+1]:
    return (xx[j+1] - x)/h
  else:
    return 0
  #return x
  
  
  #for x_ in range(n):
    #1 + (x - xx[j])/h

In [7]:
for i in range(n):
  for j in range(n):
    
    Kxi_li= lambda s: K(xx[i], s) * basicFunction(j, s)
    
    (integral, err) = integrate.quad(Kxi_li, a, b)
    
    #print(a, b, integral, basicFunction(j, xx[i]))
    
    A[i][j] = basicFunction(j, xx[i]) - integral
    
    #integral = sm.integrate(K(xx[i], y)*, (y, a, b))
    #A[i,j] = (basicFunction(j, x) - integral).evalf(5,subs={x: xx[i]})

np.round(A, 3)

<ipython-input-7-bfb191bb32af>:6: IntegrationWarning: The integral is probably divergent, or slowly convergent.
  (integral, err) = integrate.quad(Kxi_li, a, b)


array([[ 0.902, -0.19 , -0.18 , -0.17 , -0.16 , -0.15 , -0.14 , -0.13 ,
        -0.12 , -0.11 , -0.1  , -0.09 , -0.08 , -0.07 , -0.06 , -0.05 ,
        -0.04 , -0.03 , -0.02 , -0.01 , -0.002],
       [-0.084,  0.838, -0.153, -0.144, -0.135, -0.126, -0.117, -0.108,
        -0.099, -0.09 , -0.081, -0.072, -0.063, -0.054, -0.045, -0.036,
        -0.027, -0.018, -0.009,  0.   ,  0.003],
       [-0.071, -0.136,  0.872, -0.12 , -0.112, -0.104, -0.096, -0.088,
        -0.08 , -0.072, -0.064, -0.056, -0.048, -0.04 , -0.032, -0.024,
        -0.016, -0.008,  0.   ,  0.008,  0.007],
       [-0.058, -0.112, -0.105,  0.902, -0.091, -0.084, -0.077, -0.07 ,
        -0.063, -0.056, -0.049, -0.042, -0.035, -0.028, -0.021, -0.014,
        -0.007, -0.   ,  0.007,  0.014,  0.009],
       [-0.047, -0.09 , -0.084, -0.078,  0.928, -0.066, -0.06 , -0.054,
        -0.048, -0.042, -0.036, -0.03 , -0.024, -0.018, -0.012, -0.006,
        -0.   ,  0.006,  0.012,  0.018,  0.011],
       [-0.037, -0.07 , -0.065, -0.

In [8]:
ff = [[f(xx[j])] for j in range(n)]
ff

[[1],
 [1],
 [1],
 [1],
 [1],
 [1],
 [1],
 [1],
 [1],
 [1],
 [1],
 [1],
 [1],
 [1],
 [1],
 [1],
 [1],
 [1],
 [1],
 [1],
 [1]]

In [9]:
A = np.array(A, dtype='float')
np.round(A, 3)

array([[ 0.902, -0.19 , -0.18 , -0.17 , -0.16 , -0.15 , -0.14 , -0.13 ,
        -0.12 , -0.11 , -0.1  , -0.09 , -0.08 , -0.07 , -0.06 , -0.05 ,
        -0.04 , -0.03 , -0.02 , -0.01 , -0.002],
       [-0.084,  0.838, -0.153, -0.144, -0.135, -0.126, -0.117, -0.108,
        -0.099, -0.09 , -0.081, -0.072, -0.063, -0.054, -0.045, -0.036,
        -0.027, -0.018, -0.009,  0.   ,  0.003],
       [-0.071, -0.136,  0.872, -0.12 , -0.112, -0.104, -0.096, -0.088,
        -0.08 , -0.072, -0.064, -0.056, -0.048, -0.04 , -0.032, -0.024,
        -0.016, -0.008,  0.   ,  0.008,  0.007],
       [-0.058, -0.112, -0.105,  0.902, -0.091, -0.084, -0.077, -0.07 ,
        -0.063, -0.056, -0.049, -0.042, -0.035, -0.028, -0.021, -0.014,
        -0.007, -0.   ,  0.007,  0.014,  0.009],
       [-0.047, -0.09 , -0.084, -0.078,  0.928, -0.066, -0.06 , -0.054,
        -0.048, -0.042, -0.036, -0.03 , -0.024, -0.018, -0.012, -0.006,
        -0.   ,  0.006,  0.012,  0.018,  0.011],
       [-0.037, -0.07 , -0.065, -0.

In [10]:
ff = np.array(ff, dtype='float')
ff.shape

(21, 1)

In [11]:
LA.det(A)

0.1099999999895269

In [12]:
c = LA.solve(A, ff)
c

array([[7.06060606],
       [5.90909091],
       [4.87878788],
       [3.96969697],
       [3.18181818],
       [2.51515151],
       [1.96969697],
       [1.54545455],
       [1.24242424],
       [1.06060606],
       [1.        ],
       [1.06060606],
       [1.24242424],
       [1.54545455],
       [1.96969697],
       [2.51515151],
       [3.18181818],
       [3.96969697],
       [4.87878788],
       [5.90909091],
       [7.06060606]])

In [13]:
c.shape

(21, 1)

In [14]:
c.round(3)

array([[7.061],
       [5.909],
       [4.879],
       [3.97 ],
       [3.182],
       [2.515],
       [1.97 ],
       [1.545],
       [1.242],
       [1.061],
       [1.   ],
       [1.061],
       [1.242],
       [1.545],
       [1.97 ],
       [2.515],
       [3.182],
       [3.97 ],
       [4.879],
       [5.909],
       [7.061]])

In [15]:
np.dot(A, c)

array([[1.],
       [1.],
       [1.],
       [1.],
       [1.],
       [1.],
       [1.],
       [1.],
       [1.],
       [1.],
       [1.],
       [1.],
       [1.],
       [1.],
       [1.],
       [1.],
       [1.],
       [1.],
       [1.],
       [1.],
       [1.]])

In [26]:
funcs = []
for i in range(n):
  funcs.append(lambda x: c[i][0]*basicFunction(i, x))

#funcs = [lambda x: c[i][0]*basicFunction(i, x) for i in range(n)]
y_approx = lambda x: sum(f_(x) for f_ in funcs)
y_approx

<function __main__.<lambda>(x)>

In [35]:
def y_approx(x, c=c):
  return sum(c[i][0]*basicFunction(i, x) for i in range(n))

In [36]:
print(y(0.734), y_approx(0.734))

4.232536 4.2787878793818415


In [27]:
for f in funcs:
  print(inspect.getsource(f))

  funcs.append(lambda x: c[i][0]*basicFunction(i, x))

  funcs.append(lambda x: c[i][0]*basicFunction(i, x))

  funcs.append(lambda x: c[i][0]*basicFunction(i, x))

  funcs.append(lambda x: c[i][0]*basicFunction(i, x))

  funcs.append(lambda x: c[i][0]*basicFunction(i, x))

  funcs.append(lambda x: c[i][0]*basicFunction(i, x))

  funcs.append(lambda x: c[i][0]*basicFunction(i, x))

  funcs.append(lambda x: c[i][0]*basicFunction(i, x))

  funcs.append(lambda x: c[i][0]*basicFunction(i, x))

  funcs.append(lambda x: c[i][0]*basicFunction(i, x))

  funcs.append(lambda x: c[i][0]*basicFunction(i, x))

  funcs.append(lambda x: c[i][0]*basicFunction(i, x))

  funcs.append(lambda x: c[i][0]*basicFunction(i, x))

  funcs.append(lambda x: c[i][0]*basicFunction(i, x))

  funcs.append(lambda x: c[i][0]*basicFunction(i, x))

  funcs.append(lambda x: c[i][0]*basicFunction(i, x))

  funcs.append(lambda x: c[i][0]*basicFunction(i, x))

  funcs.append(lambda x: c[i][0]*basicFunction(i, x))

  funcs.ap

In [17]:
y = lambda x: 6*x**2 + 1

In [18]:
print(y(0.543), y_approx(0.543))

2.769094 0.0


In [19]:
[y_approx(x) for x in xx]

[0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 148.27272725031122]